In [1]:
import keras
from keras import models, layers

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend
from keras.datasets import mnist

In [5]:
# 데이터 획득
# mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [11]:
# 이미지 크기
img_rows, img_cols = x_train.shape[1:] # (28, 28)
img_rows, img_cols

(28, 28)

In [12]:
# 채널의 위치 조정
backend.image_data_format()

'channels_last'

In [13]:
if backend.image_data_format() == 'channel_first' :
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1 )
    x_test = x_test.reshape( x_test.shape[0], img_rows, img_cols, 1)   
    input_shape = ( 1, img_rows, img_cols )
else :
    # 흑백 이미지의 채널 정보를 처리하기 위해서는 추가 차원을 넣어야 한다
    # 컬러 이미지는 RGB 색상을 다루는 채널 정보가 이미지 데이터에 포함되어 있다
    # 흑백은 채널 정보가 없어서, 입력 데이터 차원을 하나 더 추가해야 한다
    # 채널의 위치를 확인해서 앞쪽이면 앞으로, 뒤쪽이면 뒤로 배치
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1 )
    x_test = x_test.reshape( x_test.shape[0], img_rows, img_cols, 1)
    input_shape = ( img_rows, img_cols, 1 )

In [14]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [15]:
input_shape

(28, 28, 1)

In [16]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [17]:
x_train /= 255
x_test /= 255

In [18]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [19]:
label_nums = len(set(y_train))
label_nums # 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

10

In [22]:
# 레이블을 분류형 변수로 특성 이진화 처리
y_train = keras.utils.to_categorical( y_train, label_nums )
y_test = keras.utils.to_categorical( y_test, label_nums )

In [26]:
# 이진화 처리 후
y_train[:2]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [28]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28, 1), (60000, 10), (10000, 28, 28, 1), (10000, 10))

In [29]:
# 모델 생성
model = models.Sequential()

In [ ]:
# 합성곱층 생성
# 32 : 출력 채널
# 커널 or 필터 or 가중치 → 3x3, 활성화함수 지정, 편향은 생략
model.add( layers.Conv2D( 32, kernel_size = (3, 3), 
                         activation='relu', input_shape = input_shape ) )

In [ ]:
# 풀링층
model.add( layers.MaxPool2D( pool_size = (2, 2) ) )

In [ ]:
# 과적합 방지
# 0.25 : 설정값
model.add( layers.Dropout( 0.25 ) )

In [ ]:
# 합성곱 2
model.add( layers.Conv2D( 64, (3, 3), activation='relu' ) )

In [ ]:
# 풀링층 2
model.add( layers.MaxPool2D( pool_size = (2, 2) ) )

In [ ]:
# 과적합 방지
# 0.25 : 설정값
model.add( layers.Dropout( 0.25 ) )

In [ ]:
# 데이터를 펼친다
model.add( layers.Flatten() )

In [ ]:
# 전결합층
# 128 : 출력 채널 설정
model.add( layers.Dense( 128, activation='relu' ) )

In [ ]:
# 과적합 방지
# 0.25 : 설정값
model.add( layers.Dropout( 0.5 ) )

In [ ]:
# 출력층
model.add( layers.Dense( label_nums, activation='softmax' ) )

In [ ]:
# 손실함수 : 크로스엔트로피
# 최적화 : 평균제곱근오차 : rmsprop
# 정확도
model.compile( loss = keras.losses.categorical_crossentropy,
             optimizer = 'rmsprop', metrics = ['accuracy'] )

In [ ]:
# 학습 사이즈
batch_size = 128
# 수행 횟수
epochs = 10

In [ ]:
# 훈련데이터 중에 검증용 20% 사용
his = model.fit( x_train, y_train, batch_size = batch_size, 
          epochs = epochs, validation_split = 0.2 )

In [ ]:
score = model.evaluate(x_test, y_test)
score